In [36]:
#Importing necessary libraries
import pandas as pd
import numpy as np
import pickle

In [37]:
#loading the dataset

df = pd.read_csv("T20 International Dataset.csv")

df

,Unnamed: 0,powerPlay,AverageScore,battingTeam,bowlingTeam,city,delivery_left,score,CurrentRunRate,wicketsLeft,Run_In_Last5,Wickets_In_Last5,Final_Score,innings
0,32266,0,153.142857,New Zealand,Sri Lanka,Colombo,51,94,8.173913,9,42.0,1.0,170,1
1,47001,0,158.307692,England,South Africa,Durban,0,204,10.200000,3,79.0,2.0,204,1
2,45705,0,158.307692,New Zealand,South Africa,Durban,23,79,4.886598,2,25.0,1.0,86,1
3,59293,1,149.000000,South Africa,New Zealand,Chattogram,85,39,6.685714,8,37.0,2.0,170,1
4,174494,0,137.000000,Pakistan,Sri Lanka,King City,79,44,6.439024,8,36.0,2.0,141,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
131619,104582,0,149.275862,India,New Zealand,Johannesburg,57,103,9.809524,8,27.0,2.0,180,2
131620,50938,0,168.666667,Pakistan,New Zealand,Auckland,11,136,7.486239,3,57.0,2.0,153,1
131621,80339,0,130.714286,Zimbabwe,South Africa,Hambantota,69,40,4.705882,7,34.0,1.0,93,1
131622,99444,0,128.395349,Pakistan,England,Dubai,79,37,5.414634,7,27.0,2.0,146,2


In [38]:
#Followed by Data Preprocessing

df.shape

(131624, 14)

In [39]:
df.isnull().sum()  #No NAs or null values found therefore the data is clean

Unnamed: 0          0
powerPlay           0
AverageScore        0
battingTeam         0
bowlingTeam         0
city                0
delivery_left       0
score               0
CurrentRunRate      0
wicketsLeft         0
Run_In_Last5        0
Wickets_In_Last5    0
Final_Score         0
innings             0
dtype: int64

In [40]:
df['battingTeam'].unique()

array(['New Zealand', 'England', 'South Africa', 'Pakistan', 'Bangladesh',
       'Netherlands', 'Australia', 'Zimbabwe', 'Sri Lanka', 'India',
       'West Indies', 'Ireland', 'Afghanistan'], dtype=object)

In [41]:
#Considering only the required columns for prediction

new_df = df[['battingTeam','bowlingTeam','city','score','delivery_left','wicketsLeft','CurrentRunRate','Run_In_Last5','Final_Score']]

In [42]:
new_df

,battingTeam,bowlingTeam,city,score,delivery_left,wicketsLeft,CurrentRunRate,Run_In_Last5,Final_Score
0,New Zealand,Sri Lanka,Colombo,94,51,9,8.173913,42.0,170
1,England,South Africa,Durban,204,0,3,10.200000,79.0,204
2,New Zealand,South Africa,Durban,79,23,2,4.886598,25.0,86
3,South Africa,New Zealand,Chattogram,39,85,8,6.685714,37.0,170
4,Pakistan,Sri Lanka,King City,44,79,8,6.439024,36.0,141
...,...,...,...,...,...,...,...,...,...
131619,India,New Zealand,Johannesburg,103,57,8,9.809524,27.0,180
131620,Pakistan,New Zealand,Auckland,136,11,3,7.486239,57.0,153
131621,Zimbabwe,South Africa,Hambantota,40,69,7,4.705882,34.0,93
131622,Pakistan,England,Dubai,37,79,7,5.414634,27.0,146


In [43]:
#For better prediction, considering only the cities where the matches played by the teams are more than 5.

new_df['city'].value_counts()[new_df['city'].value_counts() > 600].sum()

np.int64(116990)

In [44]:
eligible_cities = new_df['city'].value_counts()[new_df['city'].value_counts() > 600].index.tolist()

In [45]:
new_df[new_df['city'].isin(eligible_cities)]

,battingTeam,bowlingTeam,city,score,delivery_left,wicketsLeft,CurrentRunRate,Run_In_Last5,Final_Score
0,New Zealand,Sri Lanka,Colombo,94,51,9,8.173913,42.0,170
1,England,South Africa,Durban,204,0,3,10.200000,79.0,204
2,New Zealand,South Africa,Durban,79,23,2,4.886598,25.0,86
3,South Africa,New Zealand,Chattogram,39,85,8,6.685714,37.0,170
5,Bangladesh,Zimbabwe,Khulna,151,3,4,7.743590,44.0,156
...,...,...,...,...,...,...,...,...,...
131619,India,New Zealand,Johannesburg,103,57,8,9.809524,27.0,180
131620,Pakistan,New Zealand,Auckland,136,11,3,7.486239,57.0,153
131621,Zimbabwe,South Africa,Hambantota,40,69,7,4.705882,34.0,93
131622,Pakistan,England,Dubai,37,79,7,5.414634,27.0,146


In [46]:
final_df = new_df.sample(new_df.shape[0])   #Shuffling the rows for the model to be unbiased
final_df

,battingTeam,bowlingTeam,city,score,delivery_left,wicketsLeft,CurrentRunRate,Run_In_Last5,Final_Score
64050,Australia,India,Visakhapatnam,70,55,8,6.461538,30.0,127
74118,South Africa,Sri Lanka,Colombo,39,71,7,4.775510,28.0,115
101035,India,South Africa,Bengaluru,87,48,7,7.250000,25.0,134
105525,Sri Lanka,Pakistan,Hambantota,29,87,8,5.272727,21.0,99
7702,England,West Indies,London,84,59,10,8.262295,40.0,128
...,...,...,...,...,...,...,...,...,...
9811,Netherlands,Bangladesh,The Hague,59,63,6,6.210526,38.0,144
74797,New Zealand,India,Nagpur,104,10,4,5.672727,26.0,126
121735,England,South Africa,Cape Town,99,32,5,6.750000,41.0,135
41973,West Indies,Pakistan,Abu Dhabi,31,72,7,3.875000,18.0,103


In [47]:
X = final_df.drop(columns=['Final_Score'])        # Dividing the data into Train and Test
y = final_df['Final_Score']
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=1)

In [48]:
from sklearn.compose import ColumnTransformer  
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from sklearn.metrics import r2_score,mean_absolute_error

In [49]:
# Transforming columns containing text into numbers
trf = ColumnTransformer([
    ('onehot', OneHotEncoder(sparse_output=False, drop='first'), ['battingTeam', 'bowlingTeam', 'city'])
], remainder='passthrough')

In [50]:
pipe = Pipeline(steps=[
    ('step1',trf),
    ('step2',StandardScaler()),
    ('step3',XGBRegressor(n_estimators=1000,learning_rate=0.2,max_depth=12,random_state=1))
])

In [51]:
pipe.fit(X_train,y_train)
y_pred = pipe.predict(X_test)
print(r2_score(y_test,y_pred))
print(mean_absolute_error(y_test,y_pred))

0.9825551509857178
2.1435569271946227


In [52]:
pickle.dump(pipe,open('pipe.pkl','wb'))